In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import Counter, defaultdict

In [2]:
import sys
import os
def add_sys_path(p):
    p = os.path.abspath(p)
    if p not in sys.path:
        sys.path.append(p)

add_sys_path('..')

In [3]:
import evaluate

In [4]:
import data_split

In [5]:
from evaluate import read_dataset

In [6]:
import json

In [7]:
from tqdm.auto import tqdm, trange

# Чтение и сплит данных

In [8]:
n_ds = read_dataset('../data/training_data/training_nouns.tsv',  lambda x: json.loads(x))

In [9]:
len(n_ds)

25376

In [10]:
train, dev, test1, test2, hid, forbidden_words = data_split.split_dict(n_ds)

# Применение бейзлайна

In [11]:
import my_knn
from importlib import reload
reload(my_knn)

<module 'my_knn' from 'C:\\Users\\ddale\\YandexDisk\\code\\NLP\\taxonomy-enrichment\\experiments\\my_knn.py'>

In [12]:
import gensim
# this is araneum 2018 ft model from rusvectores
ft = gensim.models.fasttext.FastTextKeyedVectors.load(
    'C:/Users/ddale/Downloads/NLP/rusvectores/model.model'
)

In [13]:
ft_embedder = my_knn.SentenceEmbedder(ft=ft, n=300, normalize_word=True, pos_weights={'INFN': 1.0, 'PREP': 0.1})
print(ft_embedder('привет как дела').shape)

(300,)


In [14]:
taiga = gensim.models.KeyedVectors.load_word2vec_format(
    'C:/Users/ddale/Downloads/NLP/rusvectores/taiga_skipgram/model.bin', 
    binary=True,
)

In [15]:
w2v_embedder = my_knn.W2VWrapper(taiga)

In [16]:
w2v_embedder_pos = my_knn.W2VWrapper(taiga,  pos_weights={'NOUN': 1.0, 'PREP': 0.1}, default_weight=0.5)

In [17]:
syns_storage, rel_storage, rel_df = my_knn.prepare_storages(
    synsets_filename='../data/ruwordnet/synsets.N.xml',
    relations_filename='../data/ruwordnet/synset_relations.N.xml',
    forbidden_words=forbidden_words,
)

number of texts: 86549
forbidden senses are 4126
numer of ids 29296 long list is 95119



23827
466
5
30877


In [18]:
vecs = np.stack([ft_embedder(t) for t in tqdm(syns_storage.texts_long) ])

from sklearn.neighbors import KDTree
tree = KDTree(vecs)

In [19]:
w2v_vecs = np.stack([w2v_embedder(t) for t in tqdm(syns_storage.texts_long) ])
w2v_tree = KDTree(w2v_vecs)

In [20]:
w2v_vecs_pos = np.stack([w2v_embedder_pos(t) for t in tqdm(syns_storage.texts_long) ])
w2v_tree_pos = KDTree(w2v_vecs_pos)

In [21]:
hypos = my_knn.hypotheses_knn('брюки', index=tree, text2vec=ft_embedder, synset_storage=syns_storage, rel_storage=rel_storage)
hypos

['108194-N', '4024-N']

In [22]:
for id in hypos:
    print(syns_storage.id2synset[id]['@ruthes_name'])

ШТАНЫ, БРЮКИ
ВЕРХНЯЯ ОДЕЖДА


In [23]:
# проверка, что сравнение само с собой всегда получает скор 1
dev_pred = {k: [v0 for val in vals for v0 in val] for k, vals in dev.items()}
mean_ap, mean_rr = evaluate.get_score(dev, dev_pred, k=10)
print(mean_ap, mean_rr)

0.9996719160104987 1.0


In [24]:
class MixedIndexer:
    def __init__(self, models, vectorizers, proportions, coefs):
        self.models = models
        self.vectorizers = vectorizers
        self.proportions = proportions
        self.coefs = coefs

    def query(self, text, k=10):
        all_distances, all_indices = [] , []
        for m, v, p, c in zip(self.models, self.vectorizers, self.proportions, self.coefs):
            dis, ind = m.query(v(text).reshape(1, -1), k=max(1, int(k*p)))
            all_distances.append(dis * c)
            all_indices.append(ind)
        #return all_distances, all_indices
        return np.concatenate(all_distances, axis=1), np.concatenate(all_indices, axis=1)
        

In [25]:
indexer = MixedIndexer([w2v_tree_pos, tree], [w2v_embedder_pos, ft_embedder], [0.5, 0.5], [1, 1])

In [26]:
di, ind = indexer.query('мустанг', k=10)
di, ind

(array([[0.8238884 , 0.82399305, 0.8397748 , 0.84326875, 0.87618654,
         0.95590802, 0.96747816, 0.96927004, 0.97310478, 0.9752189 ]]),
 array([[36392, 36390, 36391, 17203,  4120,  1604, 61168,  3362,  3680,
          4192]], dtype=int64))

In [27]:
def w2v_scorer(text1, text2):
    return np.dot(w2v_embedder.get_text_vec(text1), w2v_embedder.get_text_vec(text2)) ** 5

In [28]:
#texts = ttest_test1.one_text.drop_duplicates()
texts = sorted(dev.keys())
small_hypos = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        #index=tree, text2vec=ft_embedder, 
        index=w2v_tree_pos, text2vec=w2v_embedder_pos, 
        #indexer=indexer,
        synset_storage=syns_storage, rel_storage=rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=w2v_scorer,
    )  
    for txt in tqdm(texts)
}


In [32]:
np.random.choice(list(dev.keys()))

'ТАКСОМОТОРНОЕ ПРЕДПРИЯТИЕ'

In [33]:
my_knn.hypotheses_knn(
        'ТАКСОМОТОРНОЕ ПРЕДПРИЯТИЕ', 
        #index=tree, text2vec=ft_embedder, 
        index=w2v_tree_pos, text2vec=w2v_embedder_pos, 
        #indexer=indexer,
        synset_storage=syns_storage, rel_storage=rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=w2v_scorer,
    ) 

['525-N',
 '106560-N',
 '142250-N',
 '57-N',
 '134279-N',
 '121655-N',
 '168-N',
 '2301-N',
 '4988-N',
 '138167-N']

In [51]:
np.exp(-1000)

0.0

In [60]:
reload(my_knn);

In [72]:
w2v_scorer('ТАКСОМОТОРНОЕ ПРЕДПРИЯТИЕ', 'ПРЕДПРИЯТИЕ')

0.34025797480210696

In [75]:
my_knn.hypotheses_knn(
        'ТАКСОМОТОРНОЕ ПРЕДПРИЯТИЕ', 
        #index=tree, text2vec=ft_embedder, 
        index=w2v_tree_pos, text2vec=w2v_embedder_pos, 
        #indexer=indexer,
        synset_storage=syns_storage, rel_storage=rel_storage,
        #decay=3, 
        k=100, 
        grand_mult=0.5,
        decay=3,
        neighbor_scorer=w2v_scorer,
        return_hypotheses=True,
        #verbose=True
    ).most_common(10)

[('525-N', 8.985995558529664),
 ('106560-N', 5.634369012732449),
 ('142250-N', 3.4538202566746397),
 ('57-N', 3.079341359508696),
 ('134279-N', 1.2463977341208194),
 ('121655-N', 1.156485635772695),
 ('168-N', 1.081452683496407),
 ('2301-N', 0.9988814510104708),
 ('4988-N', 0.967619097699793),
 ('138167-N', 0.944616590343989)]

In [64]:
-0.33 ** 999

-0.0

In [63]:
np.exp(-0.33 ** 999)

1.0

In [30]:
mean_ap, mean_rr = evaluate.get_score(dev, small_hypos, k=10)
print(mean_ap, mean_rr)

0.47106956682498025 0.5090129358830144


Типовой метод получает скор 26/29 на публичном тесте, и 39/42 - на дев. сете. Выглядит как непорядок. 

```
0.3964241448985541 0.428703443319584   - baseline
0.4302434851893513 0.4609392575928008  - baseline with s^2 w2v penalty 
0.4382394908969709 0.471559023872015   - with s^3 penalty
0.4441386232970875 0.478044150731158   - with s^5 penalty
0.4578544869391327 0.4919471003624545  - w2v only with s^3 penalty
0.4660694756905386 0.5044838145231845  - w2v only with s^5 penalty 
0.4661363944090322 0.5028019935008124  - w2v only with s^10 penalty 
0.4670759905011871 0.50277777777777    - w2v + ft with w2v^3 penalty [ 70 +  30 hyp, (1, 0.8) coef]
0.4669269205932591 0.5016716660417446  - w2v + ft with w2v^3 penalty [100 + 100 hyp, (1, 0.8) coef]
0.4650628306878305 0.50101393575803    - w2v + ft with w2v^3 penalty [100 + 100 hyp, (0.8, 1) coef]
0.4660975971753529 0.5008514560679912  - w2v + ft with w2v^3 penalty [100 + 100 hyp, (1,   1) coef]
0.4674433924926048 0.5024520372453443   - w2v + ft with w2v^3 penalty [ 50 +  50 hyp, (1,   1) coef]
0.4660202630921136 0.5019685039370079  - w2v with 0.1 weight for prep, 0.5 for non-nouns, 200 neighbors
0.4695603804732742 0.5073397075365577* - w2v with 0.1 weight for prep, 0.5 for non-nouns, 100 neighbors
0.4710695668249802 0.509012935883014 * - apparently the same, but why so much better?
```

### Проверка полноты

С полнотой всё НЕ ОЧЕНЬ в порядке; даже идеальное ранжирование (топ 10) даст только 65% качество. 

Если же переранжировать все топ 100, качество будет 80%. Что в принципе почти что мне подходит. 

In [93]:
small_hypos_large = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        #index=full_ft_tree, text2vec=ft_embedder, 
        index=w2v_tree, text2vec=w2v_embedder, 
        synset_storage=syns_storage, rel_storage=rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=w2v_scorer,
        result_size=100
    )  
    for txt in tqdm(dev.keys())
}

In [94]:
for k, v in small_hypos_large.items():
    gts = {sense for senses in dev[k] for sense in senses}
    small_hypos_large[k] = [sense for sense in v if sense in gts]

In [95]:
print(*evaluate.get_score(dev, small_hypos_large, k=10))

0.802985564304462 0.844488188976378


### частные случаи

In [96]:
import random

In [107]:
word = random.choice(list(dev.keys()))
print(word)

print(dev[word])
print([
    [syns_storage.id2synset[id]['@ruthes_name'] for id in ids]
    for ids in dev[word]
])

ВОЗДУШНОЕ ОТОПЛЕНИЕ
[['144-N', '145131-N', '145137-N', '5077-N']]
[['ОТОПЛЕНИЕ', 'ОБОГРЕВ ПОМЕЩЕНИЯ', 'КЛИМАТ-КОНТРОЛЬ', 'ТЕПЛОСНАБЖЕНИЕ']]


In [108]:
for h in small_hypos[word]:
    indices = [i for i, senses in enumerate(dev[word]) if h in senses]
    print(h, indices, syns_storage.id2synset[h]['@ruthes_name'])

print(evaluate.compute_ap(dev[word], small_hypos[word], 10))
print(evaluate.compute_rr([j for i in dev[word] for j in i], small_hypos[word], 10))

5583-N [] ОТОПИТЕЛЬНОЕ ОБОРУДОВАНИЕ
4435-N [] ТЕПЛОВОЕ ОБОРУДОВАНИЕ
144-N [0] ОТОПЛЕНИЕ
5077-N [0] ТЕПЛОСНАБЖЕНИЕ
145137-N [0] КЛИМАТ-КОНТРОЛЬ
145131-N [0] ОБОГРЕВ ПОМЕЩЕНИЯ
149201-N [] ИНЖЕНЕРНОЕ ОБОРУДОВАНИЕ
2070-N [] ЭЛЕКТРОТЕХНИЧЕСКОЕ ОБОРУДОВАНИЕ
145136-N [] КЛИМАТИЧЕСКОЕ ОБОРУДОВАНИЕ
4870-N [] ВЕНТИЛЯЦИОННОЕ ОБОРУДОВАНИЕ
0.3333333333333333
0.3333333333333333


Почему "слог" есть 

In [109]:
hypos = my_knn.hypotheses_knn(
    word, 
        index=w2v_tree, text2vec=w2v_embedder, 
        synset_storage=syns_storage, rel_storage=rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=w2v_scorer,
    verbose=True,
)  

0.6150570385919021 1 4874-N ВОЗДУШНЫЙ КОНДИЦИОНЕР 2
0.680940759867425 1 144-N ОТОПЛЕНИЕ 3
0.7145873508449723 1 125168-N БАТАРЕЯ ; БАТАРЕЯ ОТОПЛЕНИЕ ; БАТАРЕЯ ОТОПЛЕНИЯ ; БАТАРЕЯ ПАРОВОЙ ОТОПЛЕНИЕ ; БАТАРЕЯ ЦЕНТРАЛЬНЫЙ ОТОПЛЕНИЕ ; ОТОПИТЕЛЬНЫЙ БАТАРЕЯ ; ОТОПИТЕЛЬНЫЙ РАДИАТОР ; РАДИАТОР ; РАДИАТОР ВОДЯНОЙ ОТОПЛЕНИЕ ; РАДИАТОР ОТОПЛЕНИЕ ; РАДИАТОР ПАРОВОЙ ОТОПЛЕНИЕ ; РАДИАТОР ЦЕНТРАЛЬНЫЙ ОТОПЛЕНИЕ ; ЧУГУННЫЙ БАТАРЕЯ 1
0.717628658170076 1 2095-N ЭЛЕКТРИЧЕСКИЙ ОТОПЛЕНИЕ ; ЭЛЕКТРИЧЕСКОЕ ОТОПЛЕНИЕ 2
0.7176286603953353 1 2095-N ЭЛЕКТРИЧЕСКИЙ ОТОПЛЕНИЕ 2
0.7176286603953353 1 2095-N ЭЛЕКТРИЧЕСКОЕ ОТОПЛЕНИЕ 2
0.7236116782837223 1 125168-N РАДИАТОР ВОДЯНОЙ ОТОПЛЕНИЕ 1
0.7263033944851451 1 125168-N БАТАРЕЯ ПАРОВОЙ ОТОПЛЕНИЕ 1
0.727874659925142 1 196-N ВОЗДУШНЫЙ ТРАНСПОРТ 1
0.727874659925142 1 196-N ТРАНСПОРТ ВОЗДУШНЫЙ 1
0.7414964501770479 1 125168-N РАДИАТОР ПАРОВОЙ ОТОПЛЕНИЕ 1
0.7442230460742967 1 4914-N ВОЗДУШНЫЙ НАВИГАЦИЯ 1
0.7528918200249882 1 125112-N ВОЗДУШНЫЙ ВАННА 0
0.7528918200249882 1 125

* на "слог" вылезает какая-то муть, типа "краткий" - походу, прилетело прилагательное!

# Оценка применения

In [63]:
public_test_verbs = pd.read_csv('../data/public_test/nouns_public.tsv', header=None)
public_test_verbs.columns = ['text']

In [27]:
full_syn_storage, full_rel_storage, full_rel_df = my_knn.prepare_storages(
    synsets_filename='../data/ruwordnet/synsets.N.xml',
    relations_filename='../data/ruwordnet/synset_relations.N.xml',
    forbidden_words=set()
)

number of texts: 86549
forbidden senses are 0
numer of ids 29296 long list is 119273



28118
555
5
37249


In [28]:
full_ft_vecs = np.stack([ft_embedder(t) for t in tqdm(full_syn_storage.texts_long) ])
full_ft_tree = KDTree(full_ft_vecs)

In [64]:
full_w2v_vecs = np.stack([w2v_embedder(t) for t in tqdm(full_syn_storage.texts_long) ])
full_w2v_tree = KDTree(full_w2v_vecs)

In [171]:
full_w2v_vecs_pos = np.stack([w2v_embedder_pos(t) for t in tqdm(full_syn_storage.texts_long) ])
full_w2v_tree_pos = KDTree(full_w2v_vecs_pos)

In [176]:
public_test_hypos = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        #index=full_ft_tree, text2vec=ft_embedder, 
        index=full_w2v_tree_pos, text2vec=w2v_embedder_pos,
        synset_storage=full_syn_storage, rel_storage=full_rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=w2v_scorer,
    )  
    for txt in tqdm(public_test_verbs.text)
}

In [182]:
sub = my_knn.dict2submission(public_test_hypos, syns_storage.id2synset)
sub

,noun,result,result_text
0,АБДОМИНОПЛАСТИКА,616-N,БОЛЕЗНЬ
1,АБДОМИНОПЛАСТИКА,106665-N,"НАРУШИТЬ СОСТОЯНИЕ, ХОД"
2,АБДОМИНОПЛАСТИКА,3406-N,ПОВРЕЖДЕНИЕ ЗДОРОВЬЯ
3,АБДОМИНОПЛАСТИКА,106567-N,ФИЗИЧЕСКОЕ САМОЧУВСТВИЕ
4,АБДОМИНОПЛАСТИКА,1047-N,МЕДИЦИНА
...,...,...,...
7615,ЯТАГАН,5900-N,МЕТАЛЛИЧЕСКОЕ ИЗДЕЛИЕ
7616,ЯТАГАН,784-N,ОРУЖИЕ
7617,ЯТАГАН,106553-N,"ПРИСПОСОБЛЕНИЕ, ИНСТРУМЕНТ"
7618,ЯТАГАН,112933-N,"ОТВЕТИТЬ, ПОСТУПИТЬ В ОТВЕТ"


In [183]:
sub.to_csv('results/nouns_v4_w2v_only_simp5_pnlty_pos_weight.tsv', sep='\t', encoding='utf-8', header=None, index=None)
sub.head(15)

,noun,result,result_text
0,АБДОМИНОПЛАСТИКА,616-N,БОЛЕЗНЬ
1,АБДОМИНОПЛАСТИКА,106665-N,"НАРУШИТЬ СОСТОЯНИЕ, ХОД"
2,АБДОМИНОПЛАСТИКА,3406-N,ПОВРЕЖДЕНИЕ ЗДОРОВЬЯ
3,АБДОМИНОПЛАСТИКА,106567-N,ФИЗИЧЕСКОЕ САМОЧУВСТВИЕ
4,АБДОМИНОПЛАСТИКА,1047-N,МЕДИЦИНА
5,АБДОМИНОПЛАСТИКА,108862-N,ВОСПАЛЕНИЕ (БОЛЕЗНЕННЫЙ ПРОЦЕСС)
6,АБДОМИНОПЛАСТИКА,5804-N,КАРДИОЛОГИЯ
7,АБДОМИНОПЛАСТИКА,1064-N,ХИРУРГИЯ
8,АБДОМИНОПЛАСТИКА,6129-N,СЕРДЕЧНО-СОСУДИСТЫЕ ЗАБОЛЕВАНИЯ
9,АБДОМИНОПЛАСТИКА,2977-N,МЕДИЦИНСКОЕ ОБОРУДОВАНИЕ


# Финальная заливка

In [172]:
private_test_verbs = pd.read_csv('../data/private_test/nouns_private.tsv', header=None)
private_test_verbs.columns = ['text']

In [173]:
private_test_hypos = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        #index=full_ft_tree, text2vec=ft_embedder, 
        index=full_w2v_tree_pos, text2vec=w2v_embedder_pos,
        synset_storage=full_syn_storage, rel_storage=full_rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
        neighbor_scorer=w2v_scorer,
    )  
    for txt in tqdm(private_test_verbs.text)
}

In [179]:
sub = my_knn.dict2submission(private_test_hypos, syns_storage.id2synset)

In [180]:
sub.to_csv('results/private_nouns_v4_w2v_only_simp5_pnlty_pos_weight.tsv', sep='\t', encoding='utf-8', header=None, index=None)
sub.head(15)

,noun,result,result_text
0,АБСЕНТЕИЗМ,548-N,АЛКОГОЛЬНЫЙ НАПИТОК
1,АБСЕНТЕИЗМ,108336-N,КРЕПКИЙ АЛКОГОЛЬНЫЙ НАПИТОК
2,АБСЕНТЕИЗМ,549-N,НАПИТОК
3,АБСЕНТЕИЗМ,8367-N,ВИНО
4,АБСЕНТЕИЗМ,142362-N,СПИРТОСОДЕРЖАЩАЯ ПРОДУКЦИЯ
5,АБСЕНТЕИЗМ,3675-N,ЖИДКОСТЬ
6,АБСЕНТЕИЗМ,9419-N,ОДУРМАНИВАЮЩЕЕ ВЕЩЕСТВО
7,АБСЕНТЕИЗМ,550-N,БЕЗАЛКОГОЛЬНЫЕ НАПИТКИ
8,АБСЕНТЕИЗМ,368-N,ПРОДУКТЫ ПИТАНИЯ
9,АБСЕНТЕИЗМ,108232-N,ДЕСЕРТНОЕ ВИНО
